# GRPA Coding

Goverment Perfromance and Results Act (GPRA) coding for intake data of recovery specialist for the Institure for Prevention and Recovery (IFPR) at RWJ Baranabas

In [1]:
#importing pandas
import pandas as pd
import numpy as np

In [2]:
# importing data and parsing datetime columns
df = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\Jan GPRA\GPRA_Coding_File.csv", parse_dates = ['ADMIT_DT','FORM_DTTM',
                                                                                                'FORM_UPDATE_DTTM'])

#subsetting to include Peer Recovery Program episodes
df = df[df['EPISODE'] =='IFPR - STAR Program'].sort_values(by = ['FORM_DTTM','MRN']) # <- sorting df

In [3]:
#Renaming column to create ClientID
df.rename(columns = {'CSN':'ClientID'}, inplace = True)
df['ClientID'] = df['ClientID'].str.split(',').str[0]

In [4]:
#Creating function for interview type
def interview_type(row):
    if row['INTERVIEW_TYPE'] == 'Intake':
        return 1
    elif row['INTERVIEW_TYPE'] == '6-month follow-up':
        return 2
    elif row['INTERVIEW_TYPE'] == '3-month follow-up':
        return 4
    elif row['INTERVIEW_TYPE'] == 'Discharge':
        return 5
    
#Creating new column via applying interview_type_code function to values in interview type column
df['InterviewType'] = df.apply(interview_type, axis = 1)

## Creating GPRA 3

In [5]:
# data cleaning
df['MEDICATION_RECEIVED_FOR_STIMULANT_USE_DISORDER_PAST_30_DAYS'] = df['MEDICATION_RECEIVED_FOR_STIMULANT_USE_DISORDER_PAST_30_DAYS'].str.split(',').str[0]

#Creating function to create column StimDisContingMgmt
def Stim_Dis_Conting_Mgmt (row):
    if row['MEDICATION_RECEIVED_FOR_STIMULANT_USE_DISORDER_PAST_30_DAYS'] == 'Contingency Management':
        return 1
    else:
        return 0
    
#Creating column StimDisContingMgmt via applying the function Stim_Dis_Conting_Mgmt
df['StimDisContingMgmt'] = df.apply(Stim_Dis_Conting_Mgmt, axis = 1)

In [6]:
df.rename(columns = {'NUMBER_OF_DAYS_RECEIVED_CONTINGENCY_MANAGEMENT':'StimDisContingMgmtDays'}, inplace = True)

def StimDisContingMgmtDays (row):
    if row['StimDisContingMgmt'] == 1:
        return row['StimDisContingMgmtDays']
    
df['StimDisContingMgmtDays'] = df.apply(StimDisContingMgmtDays, axis = 1)

In [7]:
#Creating a function to create column StimDisCommReinf
def Stim_Dis_Comm_Reinf (row):
    if row['MEDICATION_RECEIVED_FOR_STIMULANT_USE_DISORDER_PAST_30_DAYS'] == 'Community Reinforcement':
        return 1
    else:
        return 0

#Creatimg column StimDisCommReinf via applying the Stim_Dis_Comm_Reinf function
df['StimDisCommReinf'] = df.apply(Stim_Dis_Comm_Reinf, axis = 1)

In [8]:
df.rename(columns = {'NUMBER_OF_DAYS_RECEIVED_COMMUNITY_REINFORCEMENT':'StimDisCommReinfDays'}, inplace = True)

def StimDisCommReinfDays (row):
    if row['StimDisCommReinf'] == 1:
        return row['StimDisCommReinfDays']
    
df['StimDisCommReinfDays'] = df.apply(StimDisCommReinfDays, axis = 1)

In [9]:
#Creating function to create column StimDisCBT
def Stim_Dis_CBT (row):
    if row['MEDICATION_RECEIVED_FOR_STIMULANT_USE_DISORDER_PAST_30_DAYS'] == 'Cognitive Behavioral Therapy':
        return 1
    else:
        return 0
    
#Creating column StimDisCBT via applying the function Stim_Dis_CBT
df['StimDisCBT'] = df.apply(Stim_Dis_CBT, axis = 1)

In [10]:
df.rename(columns = {'NUMBER_OF_DAYS_RECEIVED_COGNITIVE_BEHAVIORAL_THERAPY':'StimDisCBTDays'}, inplace = True)

def StimDisCBTDays (row):
    if row['StimDisCBT'] == 1:
        return row['StimDisCBTDays']

df['StimDisCBTDays'] = df.apply(StimDisCBTDays, axis = 1)

In [11]:
#Creating function to create column StimDisOtherEBI
def Stim_Dis_Other_EBI (row):
    if row['MEDICATION_RECEIVED_FOR_STIMULANT_USE_DISORDER_PAST_30_DAYS'] == 'Other evidence-based intervention':
        return 1
    else:
        return 0 

#Creating column StimDisOtherEBI via applying the function Stim_Dis_Other_EB
df['StimDisOtherEBI'] = df.apply(Stim_Dis_Other_EBI, axis = 1)

In [12]:
df.rename(columns = {'NUMBER_OF_DAYS_RECEIVED_OTHER_EVIDENCE_BASED_INTERVENTION':'StimDisOtherEBIDays'}, inplace = True)

def StimDisOtherEBIDays (row):
    if row['StimDisOtherEBI'] ==1:
        return row['StimDisOtherEBIDays']

df['StimDisOtherEBIDays'] = df.apply(StimDisOtherEBIDays,axis = 1)

In [13]:
#Creating function to create column StimDisNoneRcvdDiagnosed
def Stim_Dis_None_Rcvd_Diagnosed (row):
    if row['MEDICATION_RECEIVED_FOR_STIMULANT_USE_DISORDER_PAST_30_DAYS'] == 'DID NOT RECEIVE ANY INTERVENTION FOR A DIAGNOSED STIMULANT USE DISORDER':
        return 1
    else:
        return 0
    
#Creating column StimDisNoneRcvdDiagnosed via applying function  Stim_Dis_None_Rcvd_Diagnosed
df['StimDisNoneRcvdDiagnosed'] = df.apply(Stim_Dis_None_Rcvd_Diagnosed, axis = 1)

In [14]:
# reference column for StimDisNotDiagnosed
df['stimref'] = df['StimDisContingMgmt'] + df['StimDisCommReinf'] + df['StimDisCBT'] + df['StimDisOtherEBI'] + df['StimDisNoneRcvdDiagnosed']

In [15]:
#Creating function to create column StimDisNotDiagnosed
def Stim_Dis_Not_Diagnosed (row):
    if row['MEDICATION_RECEIVED_FOR_STIMULANT_USE_DISORDER_PAST_30_DAYS'] == 'CLIENT DOES NOT REPORT SUCH A DIAGNOSIS':
        return 1
    elif row['stimref'] == 0 and row['StimDisNoneRcvdDiagnosed'] == 0:
        return 1
    else:
        return 0
    
#Creating column StimDisNotDiagnosed via applying the function Stim_Dis_Not_Diagnosed
df['StimDisNotDiagnosed'] = df.apply(Stim_Dis_Not_Diagnosed, axis = 1)

In [16]:
#Replacing null values with -9
df['MEDICATION_RECEIVED_FOR_TOBACCO_USE_DISORDER_PAST_30_DAYS'] = df['MEDICATION_RECEIVED_FOR_TOBACCO_USE_DISORDER_PAST_30_DAYS'].fillna(-9)

#Creating function to create column TUDNicReplace
def TUD_Nic_Replace (row):
    if row['MEDICATION_RECEIVED_FOR_TOBACCO_USE_DISORDER_PAST_30_DAYS'] == 'Nicotine Replacement':
        return 1
    else:
        return 0

#Creating column TUDNicReplace via applying the function TUD_Nic_Replace
df['TUDNicReplace'] = df.apply(TUD_Nic_Replace, axis = 1)

In [17]:
df.rename(columns = {'NUMBER_OF_DAYS_RECEIVED_NICOTINE_REPLACEMENT':'TUDNicReplaceDays'}, inplace = True)

def TUDNicReplaceDays (row):
    if row['TUDNicReplace'] == 1:
        return row['TUDNicReplaceDays']
    
df['TUDNicReplaceDays'] = df.apply(TUDNicReplaceDays, axis = 1)

In [18]:
#Creating a function to create column TUDBupropion
def TUD_Bupropion (row):
    if row['MEDICATION_RECEIVED_FOR_TOBACCO_USE_DISORDER_PAST_30_DAYS'] == 'Bupropion':
        return 1
    else:
        return 0

#Creating column TUDBupropion via applying the function TUD_Bupropion
df['TUDBupropion'] = df.apply(TUD_Bupropion, axis = 1)

In [19]:
df.rename(columns = {'NUMBER_OF_DAYS_RECEIVED_BUPROPION':'TUDBupropionDays'}, inplace = True)

def TUDBupropionDays (row):
    if row['TUDBupropion'] == 1:
        return row['TUDBupropionDays']
    
df['TUDBupropionDays'] = df.apply(TUDBupropionDays, axis = 1)

In [20]:
#Creating a function to create column TUDVarenicline
def TUD_Varenicline (row):
    if row['MEDICATION_RECEIVED_FOR_TOBACCO_USE_DISORDER_PAST_30_DAYS'] == 'Varenicline':
        return 1
    else:
        return 0
    
#Creating column TUDVarenicline via applying the function TUD_Varenicline
df['TUDVarenicline'] = df.apply(TUD_Varenicline, axis = 1)

In [21]:
df.rename(columns = {'NUMBER_OF_DAYS_RECEIVED_VARENICLINE':'TUDVareniclineDays'}, inplace = True)

def TUDVareniclineDays (row):
    if row['TUDVarenicline'] == 1:
        return row['TUDVareniclineDays']
    
df['TUDVareniclineDays'] = df.apply(TUDVareniclineDays, axis = 1)

In [22]:
#Creating function to create column TUDNoneRcvdDiagnosed
def TUD_None_Rcvd_Diagnosed (row):
    if row['MEDICATION_RECEIVED_FOR_TOBACCO_USE_DISORDER_PAST_30_DAYS'] == 'DID NOT RECEIVE AN FDA-APPROVED MEDICATION FOR A DIAGNOSED TOBACCO USE DISORDER':
        return 1
    else:
        return 0
    
#Creating column TUDNoneRcvdDiagnosed via applying the function TUD_None_Rcvd_Diagnosed
df['TUDNoneRcvdDiagnosed'] = df.apply(TUD_None_Rcvd_Diagnosed, axis = 1)

In [23]:
# data cleaning
df['MEDICATION_RECEIVED_FOR_TOBACCO_USE_DISORDER_PAST_30_DAYS'] = df['MEDICATION_RECEIVED_FOR_TOBACCO_USE_DISORDER_PAST_30_DAYS'].str.split(',').str[0]

#Creating function to create column TUDNotDiagnosed
def TUD_Not_Diagnosed (row):
    if row['MEDICATION_RECEIVED_FOR_TOBACCO_USE_DISORDER_PAST_30_DAYS'] == 'CLIENT DOES NOT REPORT SUCH A DIAGNOSIS':
        return 1
    else:
        return 0

#Creating column TUDNotDiagnosed via applying function TUD_Not_Diagnosed
df['TUDNotDiagnosed'] = df.apply(TUD_Not_Diagnosed, axis = 1)

In [24]:
#Replacing null values with -9
df['EXPERIENCED_OVERDOSE_IN_PAST_30_DAYS'] = df['EXPERIENCED_OVERDOSE_IN_PAST_30_DAYS'].fillna(-9)

#Creating function to create column Overdose
def Overdose (x):
    if x == 'Yes (IF YES, SPECIFY BELOW IN QUESTION 7)':
        return 1
    elif x == -9:
        return -9
    elif x == 'No (IF NO, SKIP TO QUESTION 8)':
        return 0
    else:
        return -9

#Creating column Overdose via applying function Overdose
df['Overdose'] = df['EXPERIENCED_OVERDOSE_IN_PAST_30_DAYS'].apply(lambda x: Overdose (x))

In [25]:
df['Overdose'].unique()

array([ 0, -9,  1], dtype=int64)

In [26]:
#Replacing null values with -1
df['INTERVENTION_RECEIVED_AFTER_OVEROSE_IN_PAST_30_DAYS'] = df['INTERVENTION_RECEIVED_AFTER_OVEROSE_IN_PAST_30_DAYS'].fillna(-1)

#Creating function to create the column OverdoseIntervNaloxone
def Overdose_Interv_Naloxone (row):
    if row['Overdose'] in ([0,-7,-9]):
        return -1
    elif row['INTERVENTION_RECEIVED_AFTER_OVEROSE_IN_PAST_30_DAYS'] in(['Naloxone (Narcan)','Naloxone (Narcan), Supervision by someone else']):
        return 1
    else:
        return 0
    
#Creating column OverdoseIntervNaloxone via applying function Overdose_Interv_Naloxone
df['OverdoseIntervNaloxone'] = df.apply(Overdose_Interv_Naloxone,axis=1)

In [27]:
#Creating function to create column OverdoseIntervCareED
def Overdose_Interv_CareED (row):
    if row['Overdose'] in ([0,-7,-9]):
        return -1
    elif row['INTERVENTION_RECEIVED_AFTER_OVEROSE_IN_PAST_30_DAYS'] in(['Care in an Emergency Department, Naloxone (Narcan)','Care in an Emergency Department',
             'Care in an Emergency Department, Naloxone (Narcan), Other (SPECIFY), Supervision by someone else',
             'Care in an Emergency Department, REFUSED']):
        return 1
    else:
        return 0

#Creating column OverdoseIntervCareED via applying the function Overdose_Interv_CareED
df['OverdoseIntervCareED'] = df.apply(Overdose_Interv_CareED,axis=1)

In [28]:
#Creating function to create column OverdoseIntervCarePCP
def Overdose_Interv_CarePCP (row):
    if row['Overdose'] in ([0,-7,-9]):
        return -1
    elif row['INTERVENTION_RECEIVED_AFTER_OVEROSE_IN_PAST_30_DAYS'] in (['Care from a Primary Care Provider',
             'Care from a Primary Care Provider, Care in an Emergency Department, Naloxone (Narcan)']):
        return 1
    else:
        return 0
    
#Creating column OverdoseIntervCarePCP via applying the function Overdose_Interv_CarePCP
df['OverdoseIntervCarePCP'] = df.apply(Overdose_Interv_CarePCP,axis=1)

In [29]:
#Creating substring 
df['INTERVENTION_RECEIVED_AFTER_OVEROSE_IN_PAST_30_DAYS'] = df['INTERVENTION_RECEIVED_AFTER_OVEROSE_IN_PAST_30_DAYS'].str.split(',').str[0]

#Replacing null values
df['INTERVENTION_RECEIVED_AFTER_OVEROSE_IN_PAST_30_DAYS'] = df['INTERVENTION_RECEIVED_AFTER_OVEROSE_IN_PAST_30_DAYS'].fillna(-1)


#Creating function to create the column OverdoseIntervHospital
def Overdose_Interv_Hospital (row):
    if row['Overdose'] in ([0,-7,-9]):
        return -1
    elif row['INTERVENTION_RECEIVED_AFTER_OVEROSE_IN_PAST_30_DAYS'] == 'Admission to a hospital':         
        return 1
    else:
        return 0
#Creating column OverdoseIntervHospital via applying the function Overdose_Interv_Hospital
df['OverdoseIntervHospital'] = df.apply(Overdose_Interv_Hospital,axis=1)

In [30]:
#Creating function to create column OverdoseIntervSupervise
def Overdose_Interv_Supervise (row):
    if row['Overdose'] in ([0,-7,-9]):
        return -1
    elif row['INTERVENTION_RECEIVED_AFTER_OVEROSE_IN_PAST_30_DAYS'] == 'Supervision by someone else':
        return 1
    else:
        return 0

#Creating column OverdoseIntervSupervise via applying function Overdose_Interv_Supervis
df['OverdoseIntervSupervise'] = df.apply(Overdose_Interv_Supervise,axis=1)

In [31]:
#Creating function to create column OverdoseIntervOther
def Overdose_Interv_Other (row):
    if row['Overdose'] in ([0,-7,-9]):
        return -1
    elif row['INTERVENTION_RECEIVED_AFTER_OVEROSE_IN_PAST_30_DAYS'] == 'Other (SPECIFY)':
        return 1
    else:
        return 0

#Creating column OverdoseIntervOther via applying function Overdose_Interv_Other
df['OverdoseIntervOther'] = df.apply(Overdose_Interv_Other,axis=1)

In [32]:
#Renaming column 
df.rename(columns = {'OTHER_INTERVENTION':'OverdoseIntervSpec'}, inplace = True)

#Renaming column to create column OverdoseIntervSpec
df['OverdoseIntervSpec'] =  df['OverdoseIntervSpec'].str.title()

In [33]:
#Replacing null values with -9
df['HOW_MANY_TIMES_TREATED_FOR_SUBSTANCE_USE_DISORDER'] = df['HOW_MANY_TIMES_TREATED_FOR_SUBSTANCE_USE_DISORDER'].fillna(-9)

#Creating function to create column TxSUD
def Tx_SUD (x):
    if x == 'One time':
        return 1
    elif x == 'Two times':
        return 2
    elif x == 'Three times':
        return 3
    elif x == 'Four times':
        return 4
    elif x == 'Five times':
        return 5
    elif x == 'Six or more times':
        return 6
    elif x == 'Never (SKIP TO QUESTION 10)':
        return 0
    elif x == 'REFUSED (SKIP TO QUESTION 10)':
        return -7
    elif x == -9:
        return -9
    
#Creating column TxSUD via applying function Tx_SUD
df['TxSUD'] = df['HOW_MANY_TIMES_TREATED_FOR_SUBSTANCE_USE_DISORDER'].apply(lambda x:Tx_SUD (x))

In [34]:
#Replacing null values with -9
df['LAST_TIME_RECEIVED_TREATMENT_FOR_SUBSTANCE_USE_DISORDER'] = df['LAST_TIME_RECEIVED_TREATMENT_FOR_SUBSTANCE_USE_DISORDER'].fillna(-9)


#Creating function to create column TxSUDWhen
def Tx_SUD_When (x):
    if x == 'Less than 6 months ago':
        return 1
    elif x == 'Between 6 months and one year ago':
        return 2
    elif x == 'One to two years ago':
        return 3
    elif x == 'Two to three years ago':
        return 4
    elif x == 'Three to four years ago':
        return 5
    elif x == 'Five or more years ago':
        return 6
    elif x == -9:
        return -1
    elif x == 'REFUSED':
        return -7
    
#Creating column via applying the function Tx_SUD_When
df['TxSUDWhen'] = df['LAST_TIME_RECEIVED_TREATMENT_FOR_SUBSTANCE_USE_DISORDER'].apply(lambda x:Tx_SUD_When (x))    

In [35]:
#Replacing null values with -9
df['EVER_DIAGNOSED_WITH_MENTAL_HEALTH_ILLNESS'] = df['EVER_DIAGNOSED_WITH_MENTAL_HEALTH_ILLNESS'].fillna(-9)

#Creating function to create column MHDiagnosis
def MH_Diagnosis (x):
    if x == 'Yes':
        return 1
    elif x == -9:
        return -9
    elif x == 'NO (SKIP TO QUESTION 11)':
        return 0
    elif x == 'REFUSED (SKIP TO QUESTION 11)':
        return -9
    
#Creating column MHDiagnosis via applying the function MH_Diagnosis
df['MHDiagnosis'] = df['EVER_DIAGNOSED_WITH_MENTAL_HEALTH_ILLNESS'].apply(lambda x: MH_Diagnosis (x))

In [36]:
#Creating substring
df['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] = df['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'].str.split(',').str[0]

#Replacing null values with -9
df['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] = df['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'].fillna(0)

#Creating function to create column DxBriefPsych


def Dx_Brief_Psych (row):
    if row['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] == 'No Schizophrenia':
        return 0
    elif row['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] == 'Schizophrenia':
        return 1
    elif row['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Brief_Psych (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0

#Creating column DxBriefPsych via applying the function Dx_Brief_Psych
# df['DxBriefPsych'] = df['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'].apply(lambda x:Dx_Brief_Psych (x))

df['DxBriefPsych'] = df.apply(mh,axis=1)

In [37]:
#Creating function to create column DxDelusional
def Dx_Delusional (row):
    if row['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] == 'Delusional disorder':
        return 1
    elif row['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Delusional (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
    
#Creating DxDelusional column via applying function Dx_Delusional
# df['DxDelusional'] = df['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'].apply(lambda x: Dx_Delusional (x))

df['DxDelusional'] = df.apply(mh,axis=1)

In [38]:
#Creating function to create column DxSchizoaffective
def Dx_Schizoaffective (row):
    if row['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] == 'Schizoaffective disorders':
        return 1
    elif row['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Schizoaffective (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column via applying function Dx_Schizoaffective
# df['DxSchizoaffective'] = df['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'].apply(lambda x:Dx_Schizoaffective (x))

df['DxSchizoaffective'] = df.apply(mh,axis=1)

In [39]:
#Creating function to create column DxSchizophrenia
def Dx_Schizophrenia (row):
    if row['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] == 'Schizophrenia' or row['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] == 'Schizophrenia, Unspecified psychosis':
        return 1
    elif row['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] == 0:
        return 0
    else:
        return 0

def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Schizophrenia (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxSchizophrenia via applying the function Dx_Schizophrenia
# df['DxSchizophrenia'] = df['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'].apply(lambda x: Dx_Schizophrenia (x))


df['DxSchizophrenia'] = df.apply(mh,axis=1)

In [40]:
#Creating function to create column DxSchizotypal
def Dx_Schizotypal (row):
    if row['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] == 'Schizotypal':
        return 1
    elif row['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Schizotypal (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxSchizotypal via applying the function Dx_Schizotypal
df['DxSchizotypal'] = df.apply(mh,axis=1)

In [41]:
#Creating function to create column DxSharedPsychotic
def Dx_Shared_Psychotic (row): 
    if row['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] == 'Shared psychotic disorder':
        return 1
    elif row['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Shared_Psychotic (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
    
#Creating column DxSharedPsychotic via applying the function Dx_Shared_Psychotic
df['DxSharedPsychotic'] = df.apply(mh,axis=1)

In [42]:
#Creating function to create column DxUnspecPsych
def Dx_Unspec_Psych (row):
    if row['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] == 'Unspecified psychosis':
        return 1
    elif row['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] == 0:
        return 0
    else:
        return 0

def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Unspec_Psych (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0

    
#Creating column DxUnspecPsych via applying function Dx_Unspec_Psych (x)
df['DxUnspecPsych'] = df.apply(mh,axis=1)

In [43]:
#Creating function to create column DxBipolar
def Dx_Bipolar (row):
    if row['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] == 'Biploar Disorder':
        return 1
    elif row['SCHIZOPHRENIA_SCHIZOTYPAL_DELUSIONAL_OR_OTHER_NON_MOOD_PSYCHOTIC_DISORDERS'] == 0:
        return 0
    else:
        return 0

def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Bipolar (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0

    
#Creating column DxBipolar via applying function Dx_Bipolar
df['DxBipolar'] = df.apply(mh,axis=1)

In [44]:
# #Replacing null values with -9
df['MOOD_AFFECTIVE_DISORDERS'] = df['MOOD_AFFECTIVE_DISORDERS'].fillna(0)

#Creating a function to create column DxMajorDepRecurr
def Dx_Major_Dep_Recurr (row):
    if row['MOOD_AFFECTIVE_DISORDERS'] == 'Major depressive disorder, recurrent' or row['MOOD_AFFECTIVE_DISORDERS'] == 'Major depressive disorder, recurrent, Manic episode' or row['MOOD_AFFECTIVE_DISORDERS'] == 'Major depressive disorder, recurrent, No Mood [Affective] Disorders Reported':
        return 1
    elif row['MOOD_AFFECTIVE_DISORDERS'] == 'Major depressive disorder, recurrent, Major depressive disorder, single episode, Manic episode' or row['MOOD_AFFECTIVE_DISORDERS'] == 'Major depressive disorder, recurrent, Unspecified mood [affective] disorder' or row['MOOD_AFFECTIVE_DISORDERS']== 'Major depressive disorder, recurrent, Persistent mood [affective] disorders' or row['MOOD_AFFECTIVE_DISORDERS'] == 'Major depressive disorder, recurrent, Major depressive disorder, single episode':
        return 1
    elif row['MOOD_AFFECTIVE_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Major_Dep_Recurr (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0  

#Creating column DxMajorDepRecurr via applying function Dx_Major_Dep_Recurr
df['DxMajorDepRecurr'] = df.apply(mh,axis=1)

In [45]:
#Creating function to create column DxMajorDepSingle
def Dx_Major_Dep_Single (row):
    if row['MOOD_AFFECTIVE_DISORDERS'] == 'Major depressive disorder, single episode':
        return 1
    elif row['MOOD_AFFECTIVE_DISORDERS'] == 0:
        return 0
    else:
        return 0

def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Major_Dep_Single (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxMajorDepSingle via applying the function  Dx_Major_Dep_Single (x)
df['DxMajorDepSingle'] = df.apply(mh,axis=1)

In [46]:
#Creating function to create column DxManicEp
def Dx_Manic_Ep (row):
    if row['MOOD_AFFECTIVE_DISORDERS'] == 'Manic episode':
        return 1
    elif row['MOOD_AFFECTIVE_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Manic_Ep (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxManicEp via applying the function Dx_Manic_Ep (x)
df['DxManicEp'] = df.apply(mh,axis=1)

In [47]:
#Creating function to create column DxPersistMood
def Dx_Persist_Mood (row):
    if row['MOOD_AFFECTIVE_DISORDERS'] == 'Persistent mood [affective] disorders':
        return 1
    elif row['MOOD_AFFECTIVE_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Persist_Mood (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxPersistMood via applying the function Dx_Persist_Mood (x)
df['DxPersistMood'] = df.apply(mh,axis=1)

In [48]:
#Creating function to create column DxUnspecMood
def Dx_Unspec_Mood (row):
    if row['MOOD_AFFECTIVE_DISORDERS'] == 'Unspecified Mood [Affective] Disorders Reported':
        return 1
    elif row['MOOD_AFFECTIVE_DISORDERS'] == 0:
        return 0
    else:
        return 0
       
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Unspec_Mood (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxUnspecMood via applying the function Dx_Unspec_Mood (x)
df['DxUnspecMood'] = df.apply(mh,axis=1)

In [49]:
#Creating substring 
df['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] = df['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'].str.split(',').str[0]

#Replacing null values with -1
df['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] = df['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'].fillna(0)
#Creatng function to create column DxAgora
def Dx_Agora (row):
    if row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 'Agoraphobia without panic disorder':
        return 1
    elif row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Agora (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxAgora via applying the function Dx_Agora (x)
df['DxAgora'] = df.apply(mh,axis=1)

In [50]:
#Creatng function to create column DxAgoraWithPD:
def Dx_Agora_With_PD (row):
    if row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 'Agoraphobia with panic disorder':
        return 1
    elif row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Agora_With_PD (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxAgoraWithPD via applying the function Dx_Agora_With_PD (x)
df['DxAgoraWithPD'] = df.apply(mh,axis=1)

In [51]:
#Creatng function to create column DxAgoraUnspec
def Dx_Agora_Unspec (row):
    if row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 'DxAgoraUnspec':
        return 1
    elif row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Agora_Unspec (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxAgoraUnspec via applying the function Dx_Agora_Unspec (x)
df['DxAgoraUnspec'] = df.apply(mh,axis=1)

In [52]:
#Creatng function to create column DxGAD
def Dx_GAD (row):
    if row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 'Generalized anxiety disorder':
        return 1
    elif row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_GAD (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxGAD via applying the function Dx_GAD (x)
df['DxGAD'] = df.apply(mh,axis=1)

In [53]:
#Creatng function to create column DxPanic
def Dx_Panic (row):
    if row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 'Panic disorder':
        return 1
    elif row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Panic (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0

#Creating column DxPanic via applying the function Dx_Panic (x) 
df['DxPanic'] = df.apply(mh,axis=1)

In [54]:
#Creatng function to create column DxPhobic
def Dx_Phobic (row):
    if row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 'Phobic anxiety disorders':
        return 1
    elif row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Phobic (row)
    elif row['MHDiagnosis'] < 1:
        return -1
    
#Creating column DxPhobic via applying the function Dx_Phobic (x)
df['DxPhobic'] = df.apply(mh,axis =1)

In [55]:
#Creatng function to create column DxSocialPhobia 
def Dx_Social_Phobia (row):
    if row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 'Social phobias (Social anxiety disorder)':
        return 1
    elif row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 0:
        return 0
    else:
        return 0

def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Social_Phobia (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0    

#Creating column DxSocialPhobia via applying the function Dx_Social_Phobia (x)
df['DxSocialPhobia'] = df.apply(mh,axis=1)

In [56]:
#Creatng function to create column DxSpecPhobia
def Dx_Spec_Phobia (row):
    if row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 'Specific (isolated) phobias':
        return 1
    elif row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Spec_Phobia (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxSpecPhobia via applying the function Dx_Spec_Phobia (x)
df['DxSpecPhobia'] = df.apply(mh,axis=1)

In [57]:
#Creatng function to create column DxExcoriation
def Dx_Excoriation (row):
    if row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 'Excoriation (skin-picking) disorder':
        return 1
    elif row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 0:
        return 0
    else:
        return 0
    

def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Excoriation (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0    
    
#Creating column DxExcoriation via applying function Dx_Excoriation (x)
df['DxExcoriation'] = df.apply(mh,axis=1)

In [58]:
#Creatng function to create column DxHoarding
def Dx_Hoarding (row):
    if row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 'Hoarding disorder':
        return 1
    elif row['PHOBIC ANXIETY_AND_OTHER_ANXIETY_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Hoarding (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0 
    

#Creating column DxHoarding via applying the function Dx_Hoarding (x)
df['DxHoarding'] =  df.apply(mh,axis=1)

In [59]:
#Replacing null values with -9
df['OBSESSIVE_COMPULSIVE_DISORDERS'] = df['OBSESSIVE_COMPULSIVE_DISORDERS'].fillna(0)

#Creating function to create column DxOCD
def Dx_OCD (row):
    if row['OBSESSIVE_COMPULSIVE_DISORDERS'] == 'Obsessive-compulsive disorder':
        return 1
    elif row['OBSESSIVE_COMPULSIVE_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_OCD (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#creating column DxOCD via applying the function Dx_OCD (x)
df['DxOCD'] = df.apply(mh,axis=1)

In [60]:
#Creating function to create column DxOCDMixed
def Dx_OCD_Mixed (row):
    if row['OBSESSIVE_COMPULSIVE_DISORDERS'] == 'Obsessive-compulsive disorder with mixed obsessional thoughts and acts':
        return 1
    elif row['OBSESSIVE_COMPULSIVE_DISORDERS'] ==0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_OCD_Mixed (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
    
#Creating column DxOCDMixed via applying the function Dx_OCD_Mixed (x)
df['DxOCDMixed'] = df.apply(mh,axis=1)

In [61]:
#Creating substring 
df['REACTION_TO_SEVERE_STRESS_AND_ADJUSTMENT_DISORDERS'] = df['REACTION_TO_SEVERE_STRESS_AND_ADJUSTMENT_DISORDERS'].str.split(',').str[0]

#Replacing null values with -9
df['REACTION_TO_SEVERE_STRESS_AND_ADJUSTMENT_DISORDERS'] = df['REACTION_TO_SEVERE_STRESS_AND_ADJUSTMENT_DISORDERS'].fillna(0)

#Creating function to create column DxAcuteStress
def Dx_Acute_Stress (row):
    if row['REACTION_TO_SEVERE_STRESS_AND_ADJUSTMENT_DISORDERS'] == 'Acute stress disorder - reaction to severe stress':
        return 1
    elif row['REACTION_TO_SEVERE_STRESS_AND_ADJUSTMENT_DISORDERS'] == 0:
        return 0
    else:
        return 0

    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Acute_Stress (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0   

    
#Creatng column DxAcuteStress via applying the function Dx_Acute_Stress (x)
df['DxAcuteStress'] = df.apply(mh,axis=1)

In [62]:
#Creating function to create column DxAdjustment
def Dx_Adjustment (row):
    if row['REACTION_TO_SEVERE_STRESS_AND_ADJUSTMENT_DISORDERS'] == 'Adjustment disorders':
        return 1
    elif row['REACTION_TO_SEVERE_STRESS_AND_ADJUSTMENT_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Acute_Stress (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
    
#Creating column DxAdjustment via applying the fuunction  Dx_Adjustment (x)
df['DxAdjustment'] = df.apply(mh,axis=1)

In [63]:
#Creating function to create column DxBodyDysmorph
def Dx_Body_Dysmorph (row):
    if row['REACTION_TO_SEVERE_STRESS_AND_ADJUSTMENT_DISORDERS'] == 'Body dysmorphic disorder':
        return 1
    elif row['REACTION_TO_SEVERE_STRESS_AND_ADJUSTMENT_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Body_Dysmorph (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
    
#Creating column DxBodyDysmorph via applying the function Dx_Body_Dysmorph (x)
df['DxBodyDysmorph'] = df.apply(mh,axis=1)

In [64]:
#Creating function to create column DxDissocConvers
def Dx_Dissoc_Convers (row):
    if row['REACTION_TO_SEVERE_STRESS_AND_ADJUSTMENT_DISORDERS'] == 'Dissociative and conversion disorders':
        return 1
    elif row['REACTION_TO_SEVERE_STRESS_AND_ADJUSTMENT_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Dissoc_Convers (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxDissocConvers via applying the function Dx_Dissoc_Convers (x)
df['DxDissocConvers'] = df.apply(mh,axis=1)

In [65]:
#Creating function to create column DxDissocID
def Dx_Dissoc_ID (row):
    if row['REACTION_TO_SEVERE_STRESS_AND_ADJUSTMENT_DISORDERS'] == 'Dissociative identity disorder':
        return 1
    elif row['REACTION_TO_SEVERE_STRESS_AND_ADJUSTMENT_DISORDERS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Dissoc_ID (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxDissocID via applying the function Dx_Dissoc_ID (x)
df['DxDissocID'] = df.apply(mh,axis=1)

In [66]:
#Creating function to create column DxPTSD
def Dx_PTSD (row):
    if row['REACTION_TO_SEVERE_STRESS_AND_ADJUSTMENT_DISORDERS'] == 'Post traumatic stress disorder':
        return 1
    elif row['REACTION_TO_SEVERE_STRESS_AND_ADJUSTMENT_DISORDERS'] == 0:
        return 0
    else:
        return 0

def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_PTSD (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
    
#Creating column DxPTSD via applying the function Dx_PTSD (x)
df['DxPTSD'] = df.apply(mh,axis=1)

In [67]:
#Creating function to create column DxSomat
def Dx_Somat (row):
    if row['REACTION_TO_SEVERE_STRESS_AND_ADJUSTMENT_DISORDERS']  == 'Somatoform disorders':
        return 1
    elif row['REACTION_TO_SEVERE_STRESS_AND_ADJUSTMENT_DISORDERS']  == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Somat (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxSomat via applying Dx_Somat (x)
df['DxSomat'] = df.apply(mh,axis=1)

In [68]:
#Replacing null values with -9
df['BEHAVIORAL_SYNDROMES'] = df['BEHAVIORAL_SYNDROMES'].fillna(0)

#Creating function to create column DxEating
def Dx_Eating (row):
    if row['BEHAVIORAL_SYNDROMES'] == 'Eating disorders':
        return 1
    elif row['BEHAVIORAL_SYNDROMES'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Eating (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating colum DxEating via applying the function Dx_Eating (x)
df['DxEating'] = df.apply(mh,axis=1)

In [69]:
#Creating function to create column DxSleep
def Dx_Sleep (row):
    if row['BEHAVIORAL_SYNDROMES'] == 'Sleep disorders not due to a substance or known physiological condition':
        return 1
    elif row['BEHAVIORAL_SYNDROMES'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Sleep (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxSleep via applying the function 
df['DxSleep'] = df.apply(mh,axis=1)

In [70]:
#Replacing null values with -9
df['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] = df['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'].fillna(0)

#Creating function to create column DxAntisocial
def Dx_Antisocial (row):
    if row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 'Antisocial personality disorder':
        return 1
    elif row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Antisocial (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxAntisocial via applying the function Dx_Antisocial (X)
df['DxAntisocial'] = df.apply(mh,axis=1)

In [71]:
#Creating function to create column DxAvoidant
def Dx_Avoidant (row):
    if row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 'Avoidant personality disorder':
        return 1
    elif row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 0:
        return 0
    else:
        return 0

def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Avoidant (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0

    
#Creating column DxAvoidant via applying the function Dx_Avoidant (x)
df['DxAvoidant'] = df.apply(mh,axis=1)

In [72]:
#Creating function to create column DxBorderline
def Dx_Borderline (row):
    if row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 'Borderline personality disorder' or row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 'Borderline personality disorder, No Disorders of Adult Personality and Behaviors Reported':
        return 1
    elif row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 0:
        return 0
    else:
        return 0
    

def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Borderline (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
    
#Creating column DxBorderline via applying the function Dx_Borderline (x)
df['DxBorderline'] = df.apply(mh,axis=1)

In [73]:
#Creating function to create column DxDependent
def Dx_Dependent (row):
    if row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 'Dependent personality disorder':
        return 1
    elif row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 0:
        return 0
    else:
        return 0
    

def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Dependent (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxDependent via applying the function Dx_Dependent (x)
df['DxDependent'] = df.apply(mh,axis=1)

In [74]:
#Creating function to create column DxHistrionic
def Dx_Histrionic (row):
    if row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 'Histrionic personality disorder':
        return 1
    elif row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Histrionic (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0


#Creating column DxHistrionic via applying the function Dx_Histrionic (x)
df['DxHistrionic'] = df.apply(mh,axis=1)

In [75]:
#Creating function to create column DxIntellect
def Dx_Intellect (row):
    if row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 'Intellectual disabilities':
        return 1
    elif row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Intellect (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
    
#Creating column DxIntellect via applying the function Dx_Intellect (x)
df['DxIntellect'] = df.apply(mh,axis=1)

In [76]:
#Creating function to create column DxObsessCompulsPers
def Dx_Obsess_Compuls_Pers (row):
    if row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 'Obsessive-compulsive personality disorder':
        return 1
    elif row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Intellect (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxObsessCompulsPers via applying the function 
df['DxObsessCompulsPers'] = df.apply(mh,axis=1)

In [77]:
#Creating function to create column DxOtherSpecPD
def Dx_Other_SpecPD (row):
    if row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 'Other specific personality disorders':
        return 1
    elif row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Other_SpecPD (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
    
#Creating column DxOtherSpecPD via applying the function Dx_Other_SpecPD (x)
df['DxOtherSpecPD'] = df.apply(mh,axis=1)

In [78]:
#Creating function to create column DxParanoid
def Dx_Paranoid (row):
    if row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 'Paranoid personality disorder':
        return 1
    elif row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 0:
        return 0
    else:
        return 0

def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Paranoid (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0

    
#Creating column DxParanoid via applying the function Dx_Paranoid (x)
df['DxParanoid'] = df.apply(mh,axis=1)

In [79]:
#Creating function to create column DxPersonalityUnspec
def Dx_Personality_Unspec (row):
    if row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 'Personality disorder, unspecified':
        return 1
    elif row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Personality_Unspec (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0    

    
#Creating column DxPersonalityUnspec via applying function Dx_Personality_Unspec (x)
df['DxPersonalityUnspec'] = df.apply(mh,axis=1)

In [80]:
#Creating function to create column DxDevelopmentalDis
def Dx_Developmental_Dis (row):
    if row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 'Pervasive and specific developmental disorders':
        return 1
    elif row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Developmental_Dis (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxDevelopmentalDis Dx_Developmental_Dis (x)
df['DxDevelopmentalDis'] = df.apply(mh,axis=1)

In [81]:
#Creating function to create column DxSchizoidPD
def Dx_Schizoid_PD (row):
    if row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 'Schizoid personality disorder':
        return 1
    elif row['DISORDERS_OF_ADULT_PERSONALITY_AND_BEHAVIOR'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_Schizoid_PD (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating new column DxSchizoidPD via applying function Dx_Schizoid_PD (x)
df['DxSchizoidPD'] = df.apply(mh,axis=1)

In [82]:
#Replacing null values with -9
df['PATIENT_REPORTS_NO_MENTAL_HEALTH_ILLNESS'] = df['PATIENT_REPORTS_NO_MENTAL_HEALTH_ILLNESS'].fillna(0)


#Creating function to create column DxNoneOfTheAbove
def Dx_NoneOfTheAbove (row):
    if row['PATIENT_REPORTS_NO_MENTAL_HEALTH_ILLNESS'] == 'NONE OF THE ABOVE':
        return 1
    elif row['PATIENT_REPORTS_NO_MENTAL_HEALTH_ILLNESS'] == 0:
        return 0
    else:
        return 0
    
def mh (row):
    if row['MHDiagnosis'] == 1:
        return Dx_NoneOfTheAbove (row)
    elif row['MHDiagnosis'] in ([0,-9]):
        return -1
    else:
        return 0
    
#Creating column DxNoneOfTheAbove via applying the function Dx_NoneOfTheAbove (x)
df['DxNoneOfTheAbove'] = df.apply(mh,axis=1)

In [83]:
#Replacing null values with -9
df['CLIENT_SCREENED_BY_PROGRAM'] = df['CLIENT_SCREENED_BY_PROGRAM'].fillna(-9)

#Creating function to create column CooccurringScreen
def Cooccurring_Screen (row):
    if row['CLIENT_SCREENED_BY_PROGRAM'] == 'Yes':
        return 1
    elif row['CLIENT_SCREENED_BY_PROGRAM'] == -9:
        return -9
    else:
        return 0

def interview_type(row):
    if row['InterviewType'] == 1:
        return Cooccurring_Screen (row)
    else:
        pass
    
#Creating column CooccurringScreen via applying the function Cooccurring_Screen (x)
df['CooccurringScreen'] = df.apply(interview_type, axis = 1)

In [84]:
#Replace null values with -1 
df['CLIENT_SCREEN_POSITIVE_FOR_CO_OCCURRING_DISORDERS'] = df['CLIENT_SCREEN_POSITIVE_FOR_CO_OCCURRING_DISORDERS'].fillna(-1)

#Creating function for column CooccurringScreenStatus
def Cooccurring_Screen_Status (row):
    if row['CLIENT_SCREEN_POSITIVE_FOR_CO_OCCURRING_DISORDERS']== 'Yes':
        return 1
    elif row['CLIENT_SCREEN_POSITIVE_FOR_CO_OCCURRING_DISORDERS'] == 'No':
        return 0
    elif row['CLIENT_SCREEN_POSITIVE_FOR_CO_OCCURRING_DISORDERS'] == -1:
        return -1
    
def interview_type(row):
    if row['InterviewType'] == 1:
        return Cooccurring_Screen_Status (row)
    else:
        pass
    
    
#Creating column CooccurringScreenStatus via applying the function Cooccurring_Screen_Status (x)
df['CooccurringScreenStatus'] = df.apply(interview_type, axis = 1)

In [85]:
#Replacing null values with -1 
df['CLIENT_REFERRED_FOR_FURTHER_ASSESSMENT'] = df['CLIENT_REFERRED_FOR_FURTHER_ASSESSMENT'].fillna(-1)

#Creating function to create column CooccurringReferral
def Cooccurring_Referral (row):
    if row['CLIENT_REFERRED_FOR_FURTHER_ASSESSMENT'] == 'Yes':
        return 1
    elif row['CLIENT_REFERRED_FOR_FURTHER_ASSESSMENT'] == 'No':
        return 0
    elif row['CLIENT_REFERRED_FOR_FURTHER_ASSESSMENT'] == -1:
        return -1
    
def interview_type(row):
    if row['InterviewType'] == 1:
        return Cooccurring_Referral (row)
    else:
        pass

    
#Creating column CooccurringReferral via applying the function Cooccurring_Referral (x)
df['CooccurringReferral'] = df.apply(interview_type, axis = 1)

In [86]:
#Create substring 
df['MODALITY_PLANNED'] = df['MODALITY_PLANNED'].str.split(',').str[0]
df['MODALITY_PLANNED'] = df['MODALITY_PLANNED'].str.strip()

#Creating function to create column PlanSvcCaseManagement
def Plan_Svc_CaseManagement (row):
    if row['MODALITY_PLANNED'] == '1. Case Management':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 1:
        return Plan_Svc_CaseManagement (row)
    else:
        pass
    
#Creating column PlanSvcCaseManagement via applying the function Plan_Svc_CaseManagement (x)
df['PlanSvcCaseManagement'] = df.apply(interview_type, axis = 1)

In [87]:
#Creating function to create column PlanSvcIntensiveOutpatient
def Plan_Svc_Intensive_Outpatient (row):
    if row['MODALITY_PLANNED'] == '2. Intensive Outpatient Treatment':
        return 1
    else:
        return 0

def interview_type(row):
    if row['InterviewType'] == 1:
        return Plan_Svc_Intensive_Outpatient (row)
    else:
        pass
    
        
#Creating column PlanSvcIntensiveOutpatient via applying the function Plan_Svc_Intensive_Outpatient (x)
df['PlanSvcIntensiveOutpatient'] = df.apply(interview_type, axis = 1)

In [88]:
#Creating function to create column PlanSvcInpatient
def Plan_Svc_Inpatient (row):
    if row['MODALITY_PLANNED'] == '3. Inpatient/Hospital (Other Than Withdrawal Management)':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 1:
        return Plan_Svc_Inpatient (row)
    else:
        pass
    
#Creating column PlanSvcInpatient via applying the function  Plan_Svc_Inpatient (x)
df['PlanSvcInpatient'] = df.apply(interview_type, axis = 1)

In [89]:
#Creating function to create column PlanSvcOutpatient
def Plan_Svc_Outpatient (row):
    if row['MODALITY_PLANNED'] == '4. Outpatient Therapy':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 1:
        return Plan_Svc_Outpatient (row)
    else:
        pass
    
#Creating column PlanSvcOutpatient via applying the function Plan_Svc_Outpatient (x)
df['PlanSvcOutpatient'] = df.apply(interview_type, axis = 1)

In [90]:
#Creating function to create column PlanSvcOutreach
def Plan_Svc_Outreach (row):
    if row['MODALITY_PLANNED'] == '5. Outreach':
        return 1
    else:
        return 0
    
def interview_type(row):
    if row['InterviewType'] == 1:
        return Plan_Svc_Outreach (row)
    else:
        pass
    
#Creating column PlanSvcOutreachv via applying the function Plan_Svc_Outreach (x)
df['PlanSvcOutreach'] = df.apply(interview_type, axis = 1)

## Exporting GPRA 3

In [91]:
# Creating varibale for export

GPRA3_STAR = df[['ClientID','StimDisContingMgmt',	'StimDisContingMgmtDays',	'StimDisCommReinf',	'StimDisCommReinfDays',	'StimDisCBT',
                 'StimDisCBTDays',	'StimDisOtherEBI',	'StimDisOtherEBIDays',	'StimDisNoneRcvdDiagnosed',	'StimDisNotDiagnosed',
                 'TUDNicReplace',	'TUDNicReplaceDays',	'TUDBupropion',	'TUDBupropionDays',	'TUDVarenicline',
                 'TUDVareniclineDays',	'TUDNoneRcvdDiagnosed',	'TUDNotDiagnosed',	'Overdose',	'OverdoseIntervNaloxone',
                 'OverdoseIntervCareED',	'OverdoseIntervCarePCP',	'OverdoseIntervHospital',	'OverdoseIntervSupervise',
                 'OverdoseIntervOther',	'OverdoseIntervSpec',	'TxSUD',	'TxSUDWhen',	'MHDiagnosis',	'DxBriefPsych',
                 'DxDelusional',	'DxSchizoaffective',	'DxSchizophrenia',	'DxSchizotypal',	'DxSharedPsychotic',
                 'DxUnspecPsych',	'DxBipolar',	'DxMajorDepRecurr',	'DxMajorDepSingle',	'DxManicEp',	'DxPersistMood',
                 'DxUnspecMood',	'DxAgora',	'DxAgoraWithPD',	'DxAgoraUnspec',	'DxGAD',	'DxPanic',	'DxPhobic',
                 'DxSocialPhobia',	'DxSpecPhobia',	'DxExcoriation',	'DxHoarding',	'DxOCD',	'DxOCDMixed',	'DxAcuteStress',
                 'DxAdjustment',	'DxBodyDysmorph',	'DxDissocConvers',	'DxDissocID',	'DxPTSD',	'DxSomat',	'DxEating',
                 'DxSleep',	'DxAntisocial',	'DxAvoidant',	'DxBorderline',	'DxDependent',	'DxHistrionic',	'DxIntellect',
                 'DxObsessCompulsPers',	'DxOtherSpecPD',	'DxParanoid',	'DxPersonalityUnspec',	'DxDevelopmentalDis',
                 'DxSchizoidPD',	'DxNoneOfTheAbove',	'CooccurringScreen',	'CooccurringScreenStatus',	'CooccurringReferral',
                 'PlanSvcCaseManagement',	'PlanSvcIntensiveOutpatient',	'PlanSvcInpatient',	'PlanSvcOutpatient',	'PlanSvcOutreach']]

# Exporting GPRA 3
GPRA3_STAR.to_csv('GPRA3_STAR.csv', index = False, header = True, encoding = 'utf-8')